[View in Colaboratory](https://colab.research.google.com/github/mayojich/TensorFlow/blob/master/Deep%20MNIST%20for%20Experts.ipynb)

[Deep MNIST for Experts](https://www.tensorflow.org/versions/r1.2/get_started/mnist/pros)  
- Read up on ReLu

**SETUP**

**Load MNIST Data**

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

**Start TensorFlow InteractiveSession**

In [0]:
import tensorflow as tf
sess = tf.InteractiveSession()

**BUILD A SOFTMAX REGRESSION MODEL**

**Placeholders**

In [0]:
# input images and target output classes
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

**Variables**

In [0]:
# W(weights) and b(biases) are handled as variables
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [0]:
# initialize session to use all variables
sess.run(tf.global_variables_initializer())

**Predicted Class and Loss Function**

In [0]:
# regression model
y = tf.matmul(x,W) + b

In [0]:
# reduce loss. Second line applies softmax
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

**TRAIN THE MODEL**

In [0]:
# Tensorflow uses optimization algos. Gradient descent = 0.5
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [0]:
# repeat train_step in batches of 100. feed_dict replaces placeholders x and y_
for _ in range(1000):
  batch = mnist.train.next_batch(100)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

**Evaluate the Model**

In [0]:
# tf.argmax(y, 1) is label of model
# tf.argmax(y_,1) is the truth
# tf.equal checks prediction accuracy against truth
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [0]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [0]:
# accuracy of test data
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

**BUILD A MULTILAYER CONVOLUTIONAL NETWORK**

**Weight Initialization**  
Use convolutional neural network to up 92% accuracy to 99.2%.

In [0]:
# functions will run W and b. W has small noise. b is slightly +ve biased.
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

**Convolution and Pooling**

In [0]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

**First Convolutional Layer**

In [0]:
# convolution has 32 features for each 5x5 patch.
# 1d and 2d(patch size), 3d(number of input channels), 4d(number of output channels)
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [0]:
# 2d and 3d(image width and height), 4d(number of color channels)
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [0]:
# convolve x_image with the weight tensor, add the bias, apply the ReLU function
# max_pool_2x2 reduces size to 14x14
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

**Second Convolutional Layer**

In [0]:
# 64 features for each 5x5 patch
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

**Densely Connected Layer**

In [0]:
# reduce image size to 7x7. 1024 neurons to process image. 
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

# reshape tensor from pooling layer into vectors, multiple weight matrix, add bias, apply ReLu.
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

*Dropout*  
To reduce overfitting

In [0]:
# placeholder turns on dropout during training, turns off during testing
# tf.nn.dropout handles scaling neuron outputs
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

**Readout Layer**

In [0]:
# add a layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

**Train and Evaluate the Model**

In [0]:
# train and evaluate with additional stuff
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
      train_accuracy = accuracy.eval(feed_dict={
          x: batch[0], y_: batch[1], keep_prob: 1.0})
      print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

  print('test accuracy %g' % accuracy.eval(feed_dict={
      x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))